# Experiment 10 ARIMA

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 44.5 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip train.csv.zip
!unzip features.csv.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 655MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            


In [3]:
!pip install statsmodels mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# SPEED OPTIMIZATION: Suppress verbose logging globally
import logging
logging.getLogger('statsmodels').setLevel(logging.WARNING)

# Core libraries
import mlflow
import mlflow.sklearn
import dagshub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ARIMA libraries
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import joblib
import os
import json

In [5]:
class WalmartPreprocessingPipeline:
    """
    Complete preprocessing pipeline for Walmart sales data
    Supports fit/transform pattern for proper train/validation handling
    """

    def __init__(self):
        self.fitted = False
        self.outlier_thresholds = None
        self.feature_columns = None

    def load_and_prepare_data(self):
        """Load and merge train.csv, stores.csv, features.csv datasets"""
        print("📊 Loading datasets...")

        # Load datasets
        train_df = pd.read_csv('train.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")
        print(f"   🎯 Features data: {features_df.shape}")

        # Convert Date column to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        # Merge datasets
        train_stores = train_df.merge(stores_df, on='Store', how='left')
        train_full = train_stores.merge(features_df, on=['Store', 'Date'], how='left')

        print(f"   ✅ Merged data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        return train_full

    def clean_merged_data(self, train_full):
        """Clean merged data by handling duplicate IsHoliday columns"""
        print("🧹 Cleaning merged data...")

        initial_shape = train_full.shape

        # Handle duplicate IsHoliday columns if they exist
        if 'IsHoliday_x' in train_full.columns and 'IsHoliday_y' in train_full.columns:
            print("   🔄 Resolving duplicate IsHoliday columns...")
            train_full['IsHoliday'] = train_full['IsHoliday_x'] | train_full['IsHoliday_y']
            train_full = train_full.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

        print(f"   ✅ Cleaned data: {train_full.shape} (was {initial_shape})")
        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        """Create temporal split to prevent data leakage"""
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")

        # Sort by date to ensure temporal order
        df_sorted = df.sort_values('Date').reset_index(drop=True)

        # Find split point
        split_idx = int(len(df_sorted) * train_ratio)
        split_date = df_sorted.iloc[split_idx]['Date']

        # Create splits
        train_data = df_sorted.iloc[:split_idx].copy()
        val_data = df_sorted.iloc[split_idx:].copy()

        # Create split info dictionary
        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")

        return train_data, val_data, split_info

    def fit(self, train_data):
        """Fit the preprocessing pipeline on training data"""
        print("🔧 Fitting preprocessing pipeline on training data...")

        # Store training data for lag feature creation
        self.train_data_for_lags = train_data.copy()

        # Fit outlier removal thresholds on training data only
        self.outlier_thresholds = {
            'A': {'lower': -1000, 'upper': 50000},  # Type A stores
            'B': {'lower': -500, 'upper': 25000},   # Type B stores
            'C': {'lower': -200, 'upper': 15000}    # Type C stores
        }

        print("✅ Pipeline fitted on training data")
        self.fitted = True
        return self

    def transform(self, data, is_validation=False):
        """Transform data using fitted pipeline"""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transform!")

        print(f"🔄 Transforming {'validation' if is_validation else 'training'} data...")

        df = data.copy()

        # Step 1: Create date features
        df = self._create_date_features(df)

        # Step 2: Create holiday features
        df = self._create_holiday_features(df)

        # Step 3: Encode categorical features (BEFORE outlier removal!)
        df = self._encode_categorical_features(df)

        # Step 4: Create lag features (different for train vs validation)
        if is_validation:
            df = self._create_lag_features_validation(df)
        else:
            df = self._create_lag_features_training(df)

        # Step 5: Remove outliers (only on training data, AFTER encoding)
        if not is_validation:
            df = self._remove_outliers(df)

        # Step 6: Remove markdown features
        df = self._remove_markdown_features(df)

        # Step 7: Remove redundant features
        df = self._remove_redundant_features(df)

        print(f"✅ Transform complete. Shape: {df.shape}")
        return df

    def fit_transform(self, train_data):
        """Fit and transform training data in one step"""
        return self.fit(train_data).transform(train_data, is_validation=False)

    def _create_date_features(self, df):
        """Create date features"""
        df = df.copy()
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week
        df['Quarter'] = df['Date'].dt.quarter
        df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        start_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - start_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        return df

    def _create_holiday_features(self, df):
        """Create holiday features"""
        df = df.copy()
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)
        df['IsMajorHoliday'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                               df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)
        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)
        return df

    def _create_lag_features_training(self, df):
        """Create lag features for training data - DISABLED to reduce overfitting"""
        # Lag features removed to prevent overfitting
        return df

    def _create_lag_features_validation(self, df):
        """Create lag features for validation data - DISABLED to reduce overfitting"""
        # Lag features removed to prevent overfitting
        return df

    def _remove_outliers(self, df):
        """Remove outliers from training data only"""
        initial_len = len(df)
        df_clean = df.copy()

        for store_type, thresholds in self.outlier_thresholds.items():
            type_mask = df_clean[f'Type_{store_type}'] == 1
            outlier_mask = (
                (df_clean['Weekly_Sales'] < thresholds['lower']) |
                (df_clean['Weekly_Sales'] > thresholds['upper'])
            )
            df_clean = df_clean[~(type_mask & outlier_mask)]

        removed = initial_len - len(df_clean)
        print(f"   🗑️ Removed {removed:,} outliers from training data")
        return df_clean

    def _remove_markdown_features(self, df):
        """Remove markdown columns"""
        markdown_cols = [col for col in df.columns if 'MarkDown' in col]
        if markdown_cols:
            df = df.drop(markdown_cols, axis=1)
        return df

    def _remove_redundant_features(self, df):
        """Remove redundant features"""
        redundant_cols = ['Year', 'Quarter', 'Day', 'WeekOfYear', 'DaysFromStart',
                         'IsQuarterStart', 'IsQuarterEnd']
        existing_redundant = [col for col in redundant_cols if col in df.columns]
        if existing_redundant:
            df = df.drop(existing_redundant, axis=1)
        return df

    def _encode_categorical_features(self, df):
        """Encode categorical features using both one-hot and label encoding"""
        df = df.copy()

        if 'Type' in df.columns:
            print(f"   🔧 Encoding Type column using both one-hot and label encoding...")

            # One-hot encoding (existing approach)
            type_dummies = pd.get_dummies(df['Type'], prefix='Type', dtype=int)

            # Label encoding (experiment_2 approach)
            # A=0, B=1, C=2 (same as experiment_2)
            type_mapping = {'A': 0, 'B': 1, 'C': 2}
            df['Type_Encoded'] = df['Type'].map(type_mapping)

            # Add one-hot columns
            for col in type_dummies.columns:
                df[col] = type_dummies[col]

            # Remove original Type column
            df = df.drop('Type', axis=1)

            print(f"   ✅ Added both Type_Encoded and {list(type_dummies.columns)}")

        return df



In [6]:
def setup_mlflow():
    """Setup MLflow and DagsHub tracking"""
    print("🔧 Setting up MLflow and DagsHub...")

    # End any active runs first
    try:
        mlflow.end_run()
    except:
        pass

    # Initialize DagsHub
    try:
        dagshub.init(
            repo_owner='konstantine25b',
            repo_name='Walmart-Recruiting---Store-Sales-Forecasting',
            mlflow=True
        )
        print("✅ DagsHub initialized successfully!")
    except Exception as e:
        print(f"⚠️ DagsHub init warning: {e}")

    # Set MLflow tracking URI
    mlflow.set_tracking_uri("https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow")

    # Create unique experiment name with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_name = f"Experiment_10_ARIMA_{timestamp}"

    try:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"✅ Created new experiment: {experiment_name}")
    except mlflow.exceptions.MlflowException as e:
        if "already exists" in str(e):
            experiment = mlflow.get_experiment_by_name(experiment_name)
            experiment_id = experiment.experiment_id
            print(f"✅ Using existing experiment: {experiment_name}")
        else:
            # Fallback to default experiment
            experiment_name = "Default"
            mlflow.set_experiment(experiment_name)
            print(f"⚠️ Using default experiment due to: {e}")

    mlflow.set_experiment(experiment_name)

    print(f"✅ MLflow setup complete!")
    print(f"🔗 Tracking URI: {mlflow.get_tracking_uri()}")
    print(f"📊 Experiment: {experiment_name}")

    return experiment_name


In [7]:

def get_preprocessed_data():
    """
    Use preprocessing pipeline to get model-ready data

    Returns:
        X_train, y_train, X_val, y_val: Model-ready datasets
        train_holidays, val_holidays: Holiday indicators for WMAE
        split_info: Information about the temporal split
    """
    print("🔄 Getting preprocessed data using pipeline...")

    # Create the preprocessing pipeline
    pipeline = WalmartPreprocessingPipeline()

    # Load raw data
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)

    # Create temporal split
    train_data, val_data, split_info = pipeline.create_temporal_split(train_full)

    # Extract holiday information before preprocessing
    val_holidays = val_data['IsHoliday'].values.astype(bool)

    # Separate validation target (realistic test scenario)
    y_val = val_data['Weekly_Sales'].copy()
    val_data_no_target = val_data.drop('Weekly_Sales', axis=1).copy()

    # Fit and transform data using pipeline
    pipeline.fit(train_data)
    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data_no_target, is_validation=True)

    # Prepare final model data
    X_train = train_processed.drop(['Weekly_Sales', 'Date'], axis=1)
    y_train = train_processed['Weekly_Sales']
    X_val = val_processed.drop('Date', axis=1)
    train_holidays = train_processed['IsHoliday'].values.astype(bool)

    # Store feature columns for later reference
    feature_columns = list(X_train.columns)

    print(f"✅ Data preprocessing complete!")
    print(f"   📊 Training shape: {X_train.shape}")
    print(f"   📊 Validation shape: {X_val.shape}")
    print(f"   🎯 Features: {len(feature_columns)}")

    return X_train, y_train, X_val, y_val, train_holidays, val_holidays, split_info, feature_columns



In [8]:
def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
    """Calculate Weighted Mean Absolute Error (WMAE)"""
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday, holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae



In [9]:
def prepare_arima_data(X_train, y_train, X_val, y_val, train_holidays, val_holidays):
    """Prepare data for ARIMA training on Store-Dept combinations"""
    print("📊 Preparing data for ARIMA modeling...")

    # Reconstruct full datasets with dates for ARIMA
    # We need to reload the original data to get dates back
    pipeline = WalmartPreprocessingPipeline()
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full)
    train_data, val_data, _ = pipeline.create_temporal_split(train_full)

    print(f"   📈 Train data shape: {train_data.shape}")
    print(f"   📉 Val data shape: {val_data.shape}")

    # Get unique Store-Dept combinations
    train_combinations = set(zip(train_data['Store'], train_data['Dept']))
    val_combinations = set(zip(val_data['Store'], val_data['Dept']))

    print(f"   🏪 Train combinations: {len(train_combinations)}")
    print(f"   🔮 Val combinations: {len(val_combinations)}")

    # Find missing combinations in validation
    missing_in_val = train_combinations - val_combinations
    missing_in_train = val_combinations - train_combinations

    print(f"   ⚠️ Missing in validation: {len(missing_in_val)}")
    print(f"   ⚠️ Missing in training: {len(missing_in_train)}")

    return train_data, val_data, train_combinations, val_combinations



In [10]:
def auto_arima_order(ts_data, max_p=3, max_d=2, max_q=3):
    """Automatically determine ARIMA order using AIC"""
    best_aic = float('inf')
    best_order = (1, 1, 1)

    # Simple grid search for speed
    for p in range(max_p + 1):
        for d in range(max_d + 1):
            for q in range(max_q + 1):
                try:
                    model = ARIMA(ts_data, order=(p, d, q))
                    fitted_model = model.fit()
                    if fitted_model.aic < best_aic:
                        best_aic = fitted_model.aic
                        best_order = (p, d, q)
                except:
                    continue

    return best_order



In [11]:
def train_arima_models(train_data, val_data, feature_columns):
    """Train individual ARIMA models for each Store-Dept combination - OPTIMIZED FOR SPEED"""
    print("📈 Training ARIMA models for each Store-Dept combination (SPEED OPTIMIZED)...")

    # Get unique combinations from training data
    train_combinations = train_data.groupby(['Store', 'Dept']).size().index.tolist()
    print(f"   📊 Training models for {len(train_combinations)} combinations")

    models = {}
    training_errors = {}
    model_orders = {}

    successful_models = 0
    failed_models = 0

    for i, (store, dept) in enumerate(train_combinations):
        try:
            # Filter data for this combination
            store_dept_data = train_data[
                (train_data['Store'] == store) &
                (train_data['Dept'] == dept)
            ].copy()

            # Skip if insufficient data
            if len(store_dept_data) < 10:  # Need more data for ARIMA than Prophet
                failed_models += 1
                continue

            # Sort by date and create time series
            store_dept_data = store_dept_data.sort_values('Date').reset_index(drop=True)
            ts_data = store_dept_data['Weekly_Sales'].values

            # Simple differencing check for stationarity
            if adfuller(ts_data)[1] > 0.05:  # Not stationary
                # Use (1,1,1) as default for non-stationary series
                arima_order = (1, 1, 1)
            else:
                # Use (1,0,1) for stationary series
                arima_order = (1, 0, 1)

            # For some series, try auto-detection (but limit to save time)
            if len(ts_data) > 20 and successful_models % 10 == 0:  # Only for 10% of series
                arima_order = auto_arima_order(ts_data, max_p=2, max_d=1, max_q=2)

            # Fit ARIMA model
            model = ARIMA(ts_data, order=arima_order)
            fitted_model = model.fit()

            models[(store, dept)] = fitted_model
            model_orders[(store, dept)] = arima_order
            successful_models += 1

            # Quick training error calculation
            if successful_models % 50 == 0:  # Only calculate for some models to save time
                train_pred = fitted_model.fittedvalues
                train_mae = mean_absolute_error(ts_data, train_pred)
                training_errors[(store, dept)] = train_mae

            # Progress updates every 25 models
            if i % 25 == 0:
                print(f"   ✅ Trained {i+1}/{len(train_combinations)} models")

        except Exception as e:
            failed_models += 1
            if failed_models < 3:  # Only print first few errors
                print(f"   ⚠️ Failed to train model for Store {store}, Dept {dept}: {e}")

    print(f"✅ ARIMA training complete!")
    print(f"   🎯 Successful models: {successful_models}")
    print(f"   ❌ Failed models: {failed_models}")

    return models, training_errors, model_orders



In [12]:
def make_arima_predictions(models, val_data, model_orders):
    """Make predictions using trained ARIMA models"""
    print("📈 Making ARIMA predictions...")

    predictions = []
    actuals = []
    holidays = []
    successful_predictions = 0
    failed_predictions = 0

    # Get validation combinations
    val_combinations = val_data.groupby(['Store', 'Dept']).groups.keys()

    for store, dept in val_combinations:
        try:
            # Check if we have a model for this combination
            if (store, dept) not in models:
                # Use overall mean as fallback
                fallback_pred = val_data['Weekly_Sales'].mean()
                store_dept_val = val_data[
                    (val_data['Store'] == store) &
                    (val_data['Dept'] == dept)
                ]
                predictions.extend([fallback_pred] * len(store_dept_val))
                actuals.extend(store_dept_val['Weekly_Sales'].tolist())
                holidays.extend(store_dept_val['IsHoliday'].tolist())
                failed_predictions += len(store_dept_val)
                continue

            # Get validation data for this combination
            store_dept_val = val_data[
                (val_data['Store'] == store) &
                (val_data['Dept'] == dept)
            ].copy()

            # Sort by date
            store_dept_val = store_dept_val.sort_values('Date').reset_index(drop=True)

            # Make prediction using the fitted model
            model = models[(store, dept)]
            n_periods = len(store_dept_val)

            # Forecast future values
            forecast = model.forecast(steps=n_periods)

            # Store results
            predictions.extend(forecast.tolist())
            actuals.extend(store_dept_val['Weekly_Sales'].tolist())
            holidays.extend(store_dept_val['IsHoliday'].tolist())
            successful_predictions += len(store_dept_val)

        except Exception as e:
            # Fallback prediction
            fallback_pred = val_data['Weekly_Sales'].mean()
            store_dept_val = val_data[
                (val_data['Store'] == store) &
                (val_data['Dept'] == dept)
            ]
            predictions.extend([fallback_pred] * len(store_dept_val))
            actuals.extend(store_dept_val['Weekly_Sales'].tolist())
            holidays.extend(store_dept_val['IsHoliday'].tolist())
            failed_predictions += len(store_dept_val)

    print(f"✅ Predictions complete!")
    print(f"   🎯 Successful predictions: {successful_predictions}")
    print(f"   ❌ Failed/fallback predictions: {failed_predictions}")

    return np.array(predictions), np.array(actuals), np.array(holidays)



In [13]:
def main():
    """Main experiment execution"""
    print("🚀 Starting Experiment 10: ARIMA with Experiment 7 Features")
    print("=" * 80)

    # Setup MLflow tracking
    experiment_name = setup_mlflow()

    with mlflow.start_run(run_name="ARIMA_Exp7_Features_Complete") as run:
        print(f"🔄 Starting MLflow run: {run.info.run_id}")

        # Log experiment metadata
        mlflow.log_param("experiment_type", "ARIMA_with_Exp7_Features")
        mlflow.log_param("model_type", "ARIMA_Individual_Models")
        mlflow.log_param("feature_engineering", "Experiment_7_Pipeline")
        mlflow.log_param("data_split", "temporal_80_20")

        try:
            # Step 1: Get preprocessed data
            print("\n📊 Step 1: Data preprocessing...")
            X_train, y_train, X_val, y_val, train_holidays, val_holidays, split_info, feature_columns = get_preprocessed_data()

            # Log data info
            mlflow.log_metric("train_samples", len(X_train))
            mlflow.log_metric("val_samples", len(X_val))
            mlflow.log_metric("total_features", len(feature_columns))
            mlflow.log_param("split_date", str(split_info['split_date']))

            # Step 2: Prepare ARIMA-specific data
            print("\n📊 Step 2: Preparing ARIMA data...")
            train_data, val_data, train_combinations, val_combinations = prepare_arima_data(
                X_train, y_train, X_val, y_val, train_holidays, val_holidays
            )

            # Log combination info
            mlflow.log_metric("train_combinations", len(train_combinations))
            mlflow.log_metric("val_combinations", len(val_combinations))
            mlflow.log_metric("missing_combinations", len(train_combinations - val_combinations))

            # Step 3: Train ARIMA models
            print("\n📈 Step 3: Training ARIMA models...")
            models, training_errors, model_orders = train_arima_models(
                train_data, val_data, feature_columns
            )

            # Log training info
            mlflow.log_metric("successful_models", len(models))
            mlflow.log_metric("avg_training_mae", np.mean(list(training_errors.values())) if training_errors else 0)
            mlflow.log_metric("num_models", len(models))

            # Step 4: Make predictions
            print("\n📈 Step 4: Making predictions...")
            y_pred, y_true, is_holiday = make_arima_predictions(
                models, val_data, model_orders
            )

            # Step 5: Calculate metrics
            print("\n📊 Step 5: Calculating metrics...")

            # Standard metrics
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            r2 = r2_score(y_true, y_pred)

            # WMAE (Competition metric)
            wmae = calculate_wmae(y_true, y_pred, is_holiday)

            # Holiday vs non-holiday breakdown
            holiday_mask = is_holiday == True
            non_holiday_mask = is_holiday == False

            holiday_mae = mean_absolute_error(y_true[holiday_mask], y_pred[holiday_mask]) if holiday_mask.sum() > 0 else 0
            non_holiday_mae = mean_absolute_error(y_true[non_holiday_mask], y_pred[non_holiday_mask]) if non_holiday_mask.sum() > 0 else 0

            # Log all metrics
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2_score", r2)
            mlflow.log_metric("wmae", wmae)
            mlflow.log_metric("holiday_mae", holiday_mae)
            mlflow.log_metric("non_holiday_mae", non_holiday_mae)
            mlflow.log_metric("holiday_samples", holiday_mask.sum())
            mlflow.log_metric("non_holiday_samples", non_holiday_mask.sum())

            # Step 6: Results summary
            print("\n" + "=" * 60)
            print("🎯 EXPERIMENT 10 RESULTS SUMMARY")
            print("=" * 60)
            print(f"📊 Validation Metrics:")
            print(f"   WMAE (Competition Metric): ${wmae:,.2f}")
            print(f"   MAE: ${mae:,.2f}")
            print(f"   RMSE: ${rmse:,.2f}")
            print(f"   R²: {r2:.4f}")
            print(f"\n📊 Holiday Breakdown:")
            print(f"   Holiday MAE: ${holiday_mae:,.2f} ({holiday_mask.sum():,} samples)")
            print(f"   Non-Holiday MAE: ${non_holiday_mae:,.2f} ({non_holiday_mask.sum():,} samples)")
            print(f"\n📊 Model Statistics:")
            print(f"   Successful models trained: {len(models):,}")
            print(f"   Store-Dept combinations: {len(train_combinations):,}")
            print(f"   Features used: {len(feature_columns):,}")

            # Step 7: Save artifacts
            print(f"\n💾 Saving model artifacts...")

            # Save model summary
            model_summary = {
                'experiment_name': experiment_name,
                'run_id': run.info.run_id,
                'models_trained': len(models),
                'feature_columns': feature_columns,
                'model_orders': {f"{k[0]}_{k[1]}": v for k, v in model_orders.items()},
                'metrics': {
                    'wmae': wmae,
                    'mae': mae,
                    'rmse': rmse,
                    'r2': r2
                }
            }

            with open('arima_exp10_summary.json', 'w') as f:
                json.dump(model_summary, f, indent=2, default=str)

            mlflow.log_artifact('arima_exp10_summary.json')

            print("✅ Experiment 10 completed successfully!")

        except Exception as e:
            print(f"❌ Experiment failed: {e}")
            mlflow.log_param("error", str(e))
            raise

    print("\n🎉 Experiment 10: ARIMA with Experiment 7 Features - COMPLETE!")



In [14]:
if __name__ == "__main__":
    main()

🚀 Starting Experiment 10: ARIMA with Experiment 7 Features
🔧 Setting up MLflow and DagsHub...


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2b2ce913-4ee3-4373-864f-bb0a443935b2&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=50fc7b2384f65248a5f8a281dd0080a5124919d2c1ac4c3c1c12148e4d7645d4




Accessing as konstantine25b

...

   ✅ Trained 2876/3313 models
   ✅ Trained 2901/3313 models
   ✅ Trained 2926/3313 models


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


   ✅ Trained 2951/3313 models
   ✅ Trained 2976/3313 models
   ✅ Trained 3001/3313 models
   ✅ Trained 3026/3313 models
   ✅ Trained 3051/3313 models
   ✅ Trained 3076/3313 models
   ✅ Trained 3101/3313 models
   ✅ Trained 3126/3313 models
   ✅ Trained 3176/3313 models


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


   ✅ Trained 3226/3313 models
   ✅ Trained 3251/3313 models
   ✅ Trained 3276/3313 models
   ✅ Trained 3301/3313 models
✅ ARIMA training complete!
   🎯 Successful models: 3139
   ❌ Failed models: 174

📈 Step 4: Making predictions...
📈 Making ARIMA predictions...
✅ Predictions complete!
   🎯 Successful predictions: 83920
   ❌ Failed/fallback predictions: 394

📊 Step 5: Calculating metrics...

🎯 EXPERIMENT 10 RESULTS SUMMARY
📊 Validation Metrics:
   WMAE (Competition Metric): $2,589.86
   MAE: $2,546.16
   RMSE: $5,267.98
   R²: 0.9424

📊 Holiday Breakdown:
   Holiday MAE: $2,900.41 (2,966 samples)
   Non-Holiday MAE: $2,533.24 (81,348 samples)

📊 Model Statistics:
   Successful models trained: 3,139
   Store-Dept combinations: 3,313
   Features used: 25

💾 Saving model artifacts...
✅ Experiment 10 completed successfully!
🏃 View run ARIMA_Exp7_Features_Complete at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/37/runs/e96d3e3a7388454